<h1> Using Labeled Data to Train a Network, Use The Network to Label New Videos, and Create Trajectories </h1> 
<p> Use another notebook create a project, extract frames, and label training/testing data </p>

# DLC with docker
This notebook illustrates how to use the Docker container to:
- train a network
- evaluate a network
- analyze a novel video

many of the functions have additional parameters / complexity, see the DLC docs for more inf on each.

This assumes you already have a project folder with labeled data! 


## info about the Docker Environment:
Docker is essentially a better alternative to virtual machines. It is able to containerize applications in a way that keeps them seperate the OS and other software. 

For tensorflow / deeplab cut, docker is critical for two reasons. 
- It 'claims' the GPU, making it closed for use by other processes.
- It queries the GPU properly, in some cases tf (and the python kernal itself) does not know how to properly communincate with the GPU on windows systems.

In [1]:
# make sure you graphic driver is accessable
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Sep__1_21:08:32_Central_Daylight_Time_2017
Cuda compilation tools, release 9.0, V9.0.176


In [2]:
# check the tensorflow version
import tensorflow as tf
tf.__version__

'1.14.0'

<h1> The following is very important as it allow the current GPU process to grow dynamically </h1>
Without this option tf will likely run out of VRAM when trying to update the weight tensor. In theory, these options could cause the GPU to run out of memory entirely, but there is no other way to allow training to complete successfully. 

In [3]:
#allow video memory growth as network expands to avoid convolutional network errors
TF_FORCE_GPU_ALLOW_GROWTH = True
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [4]:
#let's make sure we see a GPU:
#tf.test.gpu_device_name()
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1059488224795034106,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6586089472
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17248830168826607144
 physical_device_desc: "device: 0, name: GeForce RTX 2080, pci bus id: 0000:41:00.0, compute capability: 7.5",
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 6586089472
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3503791903948238893
 physical_device_desc: "device: 1, name: GeForce RTX 2080, pci bus id: 0000:81:00.0, compute capability: 7.5"]

## Start here for training DeepLabCut and analyzing new videos!
<p><br>If the first imports fail, there is again - sadly - an issue with you enviroment. Make sure all packages beside DLC are installed via conda. </p> 

In [5]:
#GUIs don't work on in Docker (or the cloud), so label your data locally on your computer! 
#This notebook is for you to train and run video analysis!
import os

In [6]:
# now we are ready to train!
#should see version 2.0.8
import deeplabcut
deeplabcut.__version__

'2.1.10.1'

<h1> change to your path: </h1>
<p> this should be the same path as the one in the createDLCproject notebook. The path is the path to the config.yaml file, not the project directory itself </p> 

In [7]:
path_config_file = r'F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\box1_cam1-spencerloggia-2021-02-07\config.yaml'

## Create a training dataset
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

In [ ]:
deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', augmenter_type='imgaug')

### now go edit the pose_cfg.yaml to make display_iters: low (i.e. 10), and save_iters: 500

Now it is the time to start training the network!

## Start training
This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
#reset in case you started a session before...
#tf.reset_default_graph()

deeplabcut.train_network(path_config_file, shuffle=1, saveiters=1000, displayiters=10, gputouse=1)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.3M iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....


### A couple tips for possible troubleshooting (1): 

if you get **permission errors** when you run this step (above), first check if the weights downloaded. As some docker containers might not have privileges for this (it can be user specific). They should be under 'init_weights' (see path in the pose_cfg.yaml file). You can enter the DOCKER in the terminal:

see more here: https://github.com/MMathisLab/Docker4DeepLabCut2.0#using-the-docker-for-training-and-video-analysis

### Troubleshooting (2): 
if it appears the training does not start (i.e. "Starting training..." does not print immediately),
then you have another session running on your GPU. Go check "nvidia-smi" and look at the process names. You can only have 1 per GPU!)

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [8]:
deeplabcut.evaluate_network(path_config_file)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, so be sure your labels are good!

F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\box1_cam1-spencerloggia-2021-02-07/evaluation-results/  already exists!
F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\box1_cam1-spencerloggia-2021-02-07\evaluation-results\iteration-0\box1_cam1Feb7-trainset95shuffle1  already exists!
Running  DLC_resnet_50_box1_cam1Feb7shuffle1_752000  with # of trainingiterations: 752000
This net has already been evaluated!


## There is an optional refinement step
- if your pixel errors are not low enough, use DLC docs on how to refine yur network!
- You will need to adjust the labels outside of DOCKER! (you can use the createDLCproject notebook) 
-  see DLC protocol instructions on how to refine your data!

## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [9]:

VIDEO_SOURCE = r"F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\test_label"
video_all = os.listdir(VIDEO_SOURCE)
video = []
for i in range(0,len(video_all)):
    video.append(VIDEO_SOURCE + '\\' + video_all[i])
    
deeplabcut.analyze_videos(path_config_file,video)

Using snapshot-752000 for model F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\box1_cam1-spencerloggia-2021-02-07\dlc-models\iteration-0\box1_cam1Feb7-trainset95shuffle1
Initializing ResNet


  0%|                                                                                           | 0/90 [00:00<?, ?it/s]

Starting to analyze %  F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\test_label\camera-1_clip-0.avi
F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\test_label  already exists!
Loading  F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\test_label\camera-1_clip-0.avi
Duration of video [s]:  3.0 , recorded with  30.0 fps!
Overall # of frames:  90  found with (before cropping) frame dimensions:  720 720
Starting to extract posture
Cropping based on the x1 = 0 x2 = 600 y1 = 0 y2 = 450. You can adjust the cropping coordinates in the config.yaml file.


100it [00:28,  3.46it/s]                                                                                               
  0%|                                                                                           | 0/90 [00:00<?, ?it/s]

Saving results in F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\test_label...
Starting to analyze %  F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\test_label\camera-1_clip-1.avi
F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\test_label  already exists!
Loading  F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\test_label\camera-1_clip-1.avi
Duration of video [s]:  3.0 , recorded with  30.0 fps!
Overall # of frames:  90  found with (before cropping) frame dimensions:  720 720
Starting to extract posture
Cropping based on the x1 = 0 x2 = 600 y1 = 0 y2 = 450. You can adjust the cropping coordinates in the config.yaml file.


...

100it [00:01, 51.42it/s]                                                                                               
  0%|                                                                                           | 0/90 [00:00<?, ?it/s]

Saving results in F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\test_label...
Starting to analyze %  F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\test_label\camera-1_clip-99.avi
F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\test_label  already exists!
Loading  F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\test_label\camera-1_clip-99.avi
Duration of video [s]:  3.0 , recorded with  30.0 fps!
Overall # of frames:  90  found with (before cropping) frame dimensions:  720 720
Starting to extract posture
Cropping based on the x1 = 0 x2 = 600 y1 = 0 y2 = 450. You can adjust the cropping coordinates in the config.yaml file.


100it [00:01, 51.36it/s]                                                                                               


Saving results in F:\MysoreData\nbk\mouseVideoAnalysis\Box1\cam1\test_label...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


'DLC_resnet_50_box1_cam1Feb7shuffle1_752000'

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video(path_config_file,video)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook 
#for making interactive plots.
#deeplabcut.plot_trajectories(path_config_file,videofile_path, plotting=True)

deeplabcut.plot_trajectories(path_config_file,video,showfigures=True)